Goal: Violin plots of the following data

Y-axis (1 plot for each variable): Weight, Bending Stiffness (D11) above "weak layer"
    Weak Layer is: 
    - Failure layer of test (ECT, PST)
    - failure layer of avalanche (Crown, flank)
    - or identified layer of concern (Layer of concern)

X-axis: Different groups of pits
    - ECTP
    - ECTN
    - PST < 0.5
    - Pits on crowns
    - Pits on flanks
    - Layer of concern


Larger dataset
- Last five snow years


Need to code

Weight
- Density from hand hardness and grain form
- Weight above weak layer (from layer densities and thickensses)

Bending stiffness
- Elastic modulus (p 1481)
    

In [9]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from snowpylot import caaml_parser

parse_pits function: Parses all pits in a specified folder

In [10]:
def parse_pits(folder_path):
    """
    Function to parse CAAML files in the specified folder
    """

    files = [
        f for f in os.listdir(folder_path) if f.endswith(".xml")
    ]  # List of all .xml files in the folder

    pits_list = []

    for file in files:  # iterate through each file in the folder
        file_path = folder_path + "/" + file  # create the file path
        pit = caaml_parser(file_path)  # parse the file
        pits_list.append(pit)

    return pits_list

Specify folders for 2020-2024 Water Years and parse files

In [11]:
# Define folders and parse pits

pits_19_20 = parse_pits("../snowpits/by_season/2019-2020")
pits_20_21 = parse_pits("../snowpits/by_season/2020-2021")
pits_21_22 = parse_pits("../snowpits/by_season/2021-2022")
pits_22_23 = parse_pits("../snowpits/by_season/2022-2023")
pits_23_24 = parse_pits("../snowpits/by_season/2023-2024")

all_pits = (
    pits_19_20 + pits_20_21 + pits_21_22 + pits_22_23 + pits_23_24
)  # list of all pits

Geldsetzer table of density from hand hardness and grain form

In [12]:
geldsetzer_df = pd.read_csv('geldsetzer_table.csv', index_col=0)

def get_density(hand_hardness, grain_form, df=geldsetzer_df):
    """
    Get density value for a specific hand hardness and grain form combination.
    
    Parameters:
    df (pd.DataFrame): The Geldsetzer table DataFrame
    hand_hardness (str): Hand hardness value (e.g., 'F-', '4F+', 'P-', etc.)
    grain_form (str): Grain form (e.g., 'PP', 'DF', 'RG', 'FC', 'DH', etc.)
    
    Returns:
    float: Density value, or NaN if not available
    """
    try:
        return df.loc[hand_hardness, grain_form]
    except KeyError as e:
        print(f"Key not found: {e}")
        return np.nan

In [13]:
d = get_density('F-', 'PP')
print(d)

69.0


Get weight from layer and density info

NOTE: What to do about
- Layers with grain forms not in Geldsetzer table?
    - Opportunity to add to the table?
- Pits where not all of the layers above the weak layer have the info needed to calculate density

Need to check grain form, basic_grain_class_code and sub_grain_class_code?

In [45]:
pit = all_pits[2]
layers = pit.snow_profile.layers

for i, layer in enumerate(layers):
    print(f"Layer {i+1}:")
    print("Hardness:", layer.hardness, "Grain form:", layer.grain_form_primary.grain_form)

    # If layer has info needed to calculate density
    if layer.hardness is not None and layer.grain_form_primary is not None and layer.thickness is not None:
        
        #Convert grain form to code needed for Geldsetzer table
        if layer.grain_form_primary.sub_grain_class_code in ["PPgp","RGmx","FCmx"]:
            grain_form = layer.grain_form_primary.sub_grain_class_code
        else:
            grain_form = layer.grain_form_primary.basic_grain_class_code

        # Get density
        density = get_density(layer.hardness, grain_form)
        print("Density:", density, "kg/m^3")
        # Calculate weight
        thickness_m = (layer.thickness[0])/100 # convert cm to m
        weight = thickness_m * density # Weight in kg (assuming 1 m^2 cross-section)
        print("Weight:", weight, "kg")
    else:
        print("Missing info for this layer")
    
    print("")




Layer 1:
Hardness: F Grain form: SH
Key not found: 'SH'
Density: nan kg/m^3
Weight: nan kg

Layer 2:
Hardness: F Grain form: DF
Density: 101.0 kg/m^3
Weight: 14.14 kg

Layer 3:
Hardness: None Grain form: RG
Missing info for this layer

Layer 4:
Hardness: 4F Grain form: FC
Density: 204.0 kg/m^3
Weight: 10.200000000000001 kg

Layer 5:
Hardness: P Grain form: MFcr
Key not found: 'MF'
Density: nan kg/m^3
Weight: nan kg

Layer 6:
Hardness: F+ Grain form: DH
Density: 218.0 kg/m^3
Weight: 63.22 kg

Layer 7:
Hardness: P Grain form: MFcr
Key not found: 'MF'
Density: nan kg/m^3
Weight: nan kg



Find weight above indicated layer of concern

In [38]:
# Find pits with indicated layer of concern
pits_with_layer_of_concern = []

for pit in all_pits:
    layers = pit.snow_profile.layers
    for layer in layers:
        if layer.layer_of_concern:
            pits_with_layer_of_concern.append(pit)


print("Num pits: ", len(all_pits))
print("Num pits with layer of concern: ", len(pits_with_layer_of_concern))

Num pits:  31170
Num pits with layer of concern:  22545


In [41]:
# Find pits with hardness and grain form info

pits_with_hardness_and_grain_form = []

for pit in pits_with_layer_of_concern:
    all_layers_info = True # NOTE: Need to check if all layers above layer of concern, not ALL layers
    layers = pit.snow_profile.layers
    for layer in layers:
        if layer.hardness is None or layer.grain_form_primary is None:
            all_layers_info = False
    if all_layers_info: # If all layers have hardness and grain form info
        pits_with_hardness_and_grain_form.append(pit)

print("Num pits with hardness and grain form info: ", len(pits_with_hardness_and_grain_form))


Num pits with hardness and grain form info:  9316


In [47]:
results = []
for pit in pits_with_hardness_and_grain_form:
    layers = pit.snow_profile.layers
    weight_above_layer_of_concern = 0
    for layer in layers:
        #Convert grain form to code needed for Geldsetzer table
        if layer.grain_form_primary.sub_grain_class_code in ["PPgp","RGmx","FCmx"]:
            grain_form = layer.grain_form_primary.sub_grain_class_code
        else:
            grain_form = layer.grain_form_primary.basic_grain_class_code

        # Get density
        density = get_density(layer.hardness, grain_form)
        
        # Calculate weight
        thickness_m = (layer.thickness[0])/100 # convert cm to m
        weight = thickness_m * density # Weight in kg (assuming 1 m^2 cross-section)
        weight_above_layer_of_concern += weight
    # Capture info for pit
    pit_info = {
        "pit_id": pit.core_info.pit_id,
        "weight_above_layer_of_concern": weight_above_layer_of_concern
    }
    results.append(pit_info)

results_df = pd.DataFrame(results)

Key not found: 'IF'
Key not found: 'IF'
Key not found: 'MF'
Key not found: 'MF'
Key not found: 'MF'
Key not found: None
Key not found: 'IF'
Key not found: 'IF'
Key not found: 'MF'
Key not found: None
Key not found: 'IF'
Key not found: 'SH'
Key not found: 'MF'
Key not found: 'MF'
Key not found: 'MF'
Key not found: 'MF'
Key not found: 'MF'
Key not found: 'SH'
Key not found: 'MF'
Key not found: 'IF'
Key not found: None
Key not found: 'MF'
Key not found: 'IF'
Key not found: 'MF'
Key not found: 'MF'
Key not found: 'IF'
Key not found: 'MF'
Key not found: 'MF'
Key not found: 'IF'
Key not found: 'MF'
Key not found: 'MF'
Key not found: 'MF'
Key not found: 'MF'
Key not found: 'MF'
Key not found: 'MF'
Key not found: 'IF'
Key not found: 'IF'
Key not found: 'IF'
Key not found: 'MF'
Key not found: 'MF'
Key not found: 'IF'
Key not found: 'MF'
Key not found: 'MF'
Key not found: 'MF'
Key not found: 'MF'
Key not found: 'MF'
Key not found: 'IF'
Key not found: 'MF'
Key not found: 'IF'
Key not found: 'SH'


In [51]:
non_na_results = results_df[results_df['weight_above_layer_of_concern'].notna()]
print("num pits with weight above layer of concern: ", len(non_na_results))

print("mean weight above layer of concern: ", non_na_results['weight_above_layer_of_concern'].mean())

print("median weight above layer of concern: ", non_na_results['weight_above_layer_of_concern'].median())



num pits with weight above layer of concern:  2207
mean weight above layer of concern:  218.10183008608973
median weight above layer of concern:  204.26999999999998
